In [1]:
import numpy as np

#moleculas mass dictionary for atoms, add more atoms if neccesary

MM = {
    'H': 1.00784,
    'C': 12.0107,
    'O': 15.999,
    'N': 14.0067}

In [2]:
#read xyz file and stores data in 4 different arrays for atom, x, y and z coordinates for eachs atom

def read_xyz(xyz):
    
    with open(xyz,'r') as file:
        
        lines = file.readlines()[2:] #starts reading after number of atoms and comment rows
        atoms=[]
        x=[]
        y=[]
        z=[]
        
        for line in lines: #for every line in the file stores x,y and z values
            line.strip() #separetes columns omitting blanck spaces (sets it nicely to be stored)
            atoms.append(line.split()[0]) #makes columns easy to work with
            x.append(line.split()[1]),
            y.append(line.split()[2])
            z.append(line.split()[3])
            
        x = np.array(x, dtype=float) #converts to numpy arrays
        y = np.array(y, dtype=float)
        z = np.array(z, dtype=float)
        
    return atoms,x,y,z

In [3]:
#computes mass center for x,y and z coordinates, then traslates

def cm_tras(xyz):

    atoms,x,y,z = read_xyz(xyz)
    mass_sum=0
    mx=0
    my=0
    mz=0

    for i in range(len(atoms)): #runs through number of lines (atoms) to index x,y and z values for each atom
        mx += MM[atoms[i]] * x[i] #numerator for x component of CM
        my += MM[atoms[i]] * y[i]
        mz += MM[atoms[i]] * z[i]
        mass_sum += MM[atoms[i]] #calculates denominator
    
    cm = np.array([mx/mass_sum , my/mass_sum , mz/mass_sum])
    coords = np.column_stack((x,y,z)) #stacks colums to obtain 2D array with the coordinates of each atom at each subarray
    return coords - cm #substracts mass center to obtain coordinates centered in CM


In [4]:
# defines an empty 3x3 matrix for the inertia tensor and calculates every term of it

def inertia_tensor(xyz):
    
    atoms,x,y,z = read_xyz(xyz)
    tensor = np.zeros((3, 3))
    
    for i in range(len(atoms)): #analog to center fo mass loop
        
        m_i = MM[atoms[i]] #with the name of the atom store in atoms array, extracts its molar mass from dictionary
        
        tensor[0, 0] += m_i * (y[i]**2 + z[i]**2)
        tensor[1, 1] += m_i * (x[i]**2 + z[i]**2)
        tensor[2, 2] += m_i * (x[i]**2 + y[i]**2)
        tensor[0, 1] -= m_i * x[i] * y[i]
        tensor[0, 2] -= m_i * x[i] * z[i]
        tensor[1, 2] -= m_i * y[i] * z[i]
    
    tensor[1, 0] = tensor[0, 1]
    tensor[2, 0] = tensor[0, 2]
    tensor[2, 1] = tensor[1, 2]
    
    return tensor

In [5]:
#obtains tensor of inertia and dianalizes it obtaines axes and momenta of inertia

def momenta_axes(xyz):
    
    tensor = inertia_tensor(xyz)
    evals, evecs = np.linalg.eigh(tensor) #diagonalizes matrix obtaining eigenvalues and eigenvectors, eigh prepares values in acending order
    sorted_evals = evals[::-1] #preapres them in descendet, setting the principal axis at first place
    idx = np.argsort(sorted_evals) 
    sorted_evecs = evecs[:, idx] #sorts eigenvectors according to the principal axes of inertia

    rot_mat = sorted_evecs.T #raotion matrix made of eigenvectors gets traspose
    
    return evals, rot_mat

In [6]:
#rorates matrix according to eigenvector matrix and displays it in xyz format

def rotate_inertia(xyz,molecule):

    atoms,x,y,z = read_xyz(xyz)
    cm_coords = cm_tras(xyz)
    tensor = inertia_tensor(xyz)
    evals,rot_mat = momenta_axes(xyz)
    rot_coords = np.dot(cm_coords, rot_mat) #dot product of rotation matrix and mass centered coordinates

    print(len(atoms))
    print(molecule)
    for atom, coord in zip(atoms, rot_coords): #runs over elements in atom array and coordinates t print then in xyz format
        print(f"{atom} {coord[0]:.6f} {coord[1]:.6f} {coord[2]:.6f}")
    
    with open('benzene-rot.xyz', 'w') as f:
        
        #Write some text to the file
        f.write(str(len(atoms)))
        f.write("\n")
        f.write(str(molecule))
        f.write("\n")
        for atom, coord in zip(atoms, rot_coords):
            f.write(f"{atom} {coord[0]:.6f} {coord[1]:.6f} {coord[2]:.6f}\n")


rotate_inertia('benzene.xyz','benzene')

12
benzene
C 1.402720 0.000000 -0.000000
H 2.490290 0.000000 -0.000000
C 0.701360 0.000000 -1.214790
H 1.245150 0.000000 -2.156660
C -0.701360 0.000000 -1.214790
H -1.245150 0.000000 -2.156660
C -1.402720 0.000000 -0.000000
H -2.490290 0.000000 -0.000000
C -0.701360 0.000000 1.214790
H -1.245150 0.000000 2.156660
C 0.701360 0.000000 1.214790
H 1.245150 0.000000 2.156660


In [10]:
inertia_tensor('h2o-rot.xyz')

array([[1.77129103, 0.        , 0.        ],
       [0.        , 1.15035805, 0.        ],
       [0.        , 0.        , 0.62093298]])